### Análisis de mercado: Eroski price scraping

Librerias principalas utilizadas: Selenium y pandas

In [ ]:
#pip install selenium

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd 
import time

Cargamos el driver de Chrome

In [2]:
#path="C:\Users\mdorronsoro\Downloads\chromedriver_win32\chromedriver.exe"
driver=webdriver.Chrome(executable_path=r"C:\Users\mdorronsoro\Downloads\chromedriver_win32\chromedriver.exe")

C:\Users\mdorronsoro\AppData\Local\Temp\ipykernel_10808\1049481939.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(executable_path=r"C:\Users\mdorronsoro\Downloads\chromedriver_win32\chromedriver.exe")


Log in function 

In [3]:
def log_in():
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "login-desktop"))).click()
    user="maiderdorron@gmail.com"
    psw="Flavia2023!"
    user_input=wait.until(EC.element_to_be_clickable((By.ID, "email"))).send_keys(user)
    psw_input=wait.until(EC.element_to_be_clickable((By.ID, "password"))).send_keys(psw)
    wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@title='Entrar']"))).click()
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.primary-button.primary-button-back"))).click()
    time.sleep(1)

We get the main info from the products in this method <br> It is remarkable that we are executing a script which allow us to scroll down and wait until all the products are loaded

In [4]:
prices = []
items = []
urls = []
precio_kilo = []
cod_item_pos = []
cat_1 = []
cat_2 = []
SCROLL_PAUSE_TIME = 3


def look_up_it(lista):
    for i in lista:
        search=driver.find_element(By.ID, value="searchTerm")
        search.send_keys(i)
        time.sleep(0.5)
        driver.find_element(By.CLASS_NAME, "search-button").click()
        time.sleep(3)
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  
            last_height = new_height
        
        get_items_prices()
  

It gets all the products available for the search and gets the genera info. 

In [5]:
def get_items_prices():
    # precio   
    get_price = driver.find_elements(By.CLASS_NAME, "price-offer-now")
    for t in get_price:
        prices.append(t.text)

    #items
    get_products = driver.find_elements(By.CSS_SELECTOR, "h2.product-title.product-title-resp")
    for t in get_products:
        items.append(t.text)
    
    #urls    
    get_urls=driver.find_elements(By.CSS_SELECTOR, "h2.product-title a")
    for element in get_urls:
          urls.append(element.get_attribute("href"))
    

After an overview of all the available products, we are clicking on each product link in order to get all the products details

In [6]:
def get_price_kilo(urls):
    for element in urls:
        driver.get(element)
        categorias=driver.find_elements(By.CSS_SELECTOR, "div.col.col-lg-12.m__breadcrumb__path a")
        cat_1.append(categorias[1].text)
        cat_2.append(categorias[2].text)

        link = None
        try:
            link = driver.find_element(By.CLASS_NAME, "price-product")
            precio_kilo.append(link.text)
        except NoSuchElementException:
            precio_kilo.append('NA')
        #driver.close()

We extract the internal codes from the url

In [7]:
def internal_codes(urls):
    get_internal_codes= [x[49:] for x in urls]
    for element in get_internal_codes:
        splited=element.split('-') 
        cod_item_pos.append(splited[0])

In [8]:
#print(driver.title)
driver.get("https://supermercado.eroski.es/es/")
driver.maximize_window()
wait = WebDriverWait(driver, 30)
wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))).click() 
log_in()

look_up=['gula','palito cangrejo','salmon','bacalao']
look_up_it(look_up)
urls=list(set(urls))
get_price_kilo(urls)
internal_codes(urls)
driver.get("https://supermercado.eroski.es/")

In [9]:
d = {'Item' : items, 'Price':prices,'Precio_kilo':precio_kilo, 'Internal code':cod_item_pos, 'Categoria 1': cat_1,
     'Categoria 2': cat_2,'urls':urls}
df = pd.DataFrame(data=d)
df.to_csv("Eroski.csv")

In [ ]:
driver.close()
driver.quit()

All the libraries needed

In [ ]:
pip freeze